# Sequence Modeling

In this notebook, we will look at a character-level language model based on the PyTorch documentation. Bundled with this document should be a file called `docs.txt` which contains all of the documentation for PyTorch. We will learn to predict one character at a time based on a set of characters that come before it in the documentation.

In [10]:
%pylab inline
import torch
docs = open('docs.txt').read()
char_set = np.unique(list(docs))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device = ', device)
print('Unique characters:', len(char_set))

Populating the interactive namespace from numpy and matplotlib
device =  cuda
Unique characters: 101


Let's take a look at what our data looks like:

In [4]:
print(docs[:400])

torch.__config__

.. automodule:: torch.__config__

.. autofunction:: show
.. autofunction:: parallel_info
.. note::

    When using the CUDA backend, this operation may induce nondeterministic
    behaviour that is not easily switched off.
    Please see the notes on :doc:`/notes/randomness` for background.
torch.utils.checkpoint

.. note


We can see that the documentation has some markup together with text describing different PyTorch objects.

Now let's see how we can manipulate this data. First, we will define a one-hot encoding for these characters. Here each character is represented by a vector. The length of the vector is equal to the number of unique characters in the documentation, and each character is associated with one index in the vector. The character is encoded by setting the appropriate index to one and all other indices to zero.

In [7]:
one_hot = torch.as_tensor(np.array(list(docs))[None,:] == np.array(char_set)[:,None]).float()

print(docs[:3])
print(one_hot[:,:3])

tor
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
      

Now let's define a temporal convolutional network to manipulate this data.

In [8]:
class TCN(torch.nn.Module):
    def __init__(self, layers=[32,64,128,256]):
        super().__init__()
        # The number of input channels is going to be the number of characters
        # we have. This is the same as the length of each vector in a one-hot
        # encoding.
        c = len(char_set)
        L = []
        # We will use a dilation to increase the size of our receptive field.
        total_dilation = 1
        for l in layers:
            # This ConstantPad1d layer transforms our regular convolutions into
            # causal convolutions by shifting the input.
            L.append(torch.nn.ConstantPad1d((2*total_dilation,0), 0))
            L.append(torch.nn.Conv1d(c, l, 3, dilation=total_dilation))
            L.append(torch.nn.ReLU())
            # Use a dilation of 2^k in layer k
            total_dilation *= 2
            c = l
        self.network = torch.nn.Sequential(*L)
        # Finally, we have a classifier layer which produces a distribution
        # over characters. Notice that we use a convolutional layer with a
        # kernel size of 1 to produce one output per input character (our
        # spatial dimension has not changed because we used dilation instead
        # of striding).
        self.classifier = torch.nn.Conv1d(c, len(char_set), 1)

    def forward(self, x):
        return self.classifier(self.network(x))

tcn = TCN()

In [9]:
tcn(one_hot[None,:,:100]).shape

torch.Size([1, 101, 100])

In [11]:
%load_ext tensorboard
import tempfile
log_dir = tempfile.mkdtemp()
%tensorboard --logdir {log_dir} --reload_interval 1

Now we'll define a handy utility function to generate batches of data for us.

In [12]:
def make_random_batch(batch_size, seq_len):
    B = []
    for i in range(batch_size):
        # First we sample a starting position
        s = np.random.choice(one_hot.size(1)-seq_len)
        # Then we add a sequence of length seq_len starting from s to the batch
        B.append(one_hot[:,s:s+seq_len])
    return torch.stack(B, dim=0)

Now we're ready to write some training code. The interesting thing here is how we generate data.

In [15]:
import torch.utils.tensorboard as tb
from tqdm.notebook import tqdm
n_iterations = 10000
batch_size = 128
seq_len = 256

logger = tb.SummaryWriter(log_dir+'/tcn1', flush_secs=1)

tcn = TCN().to(device)

optimizer = torch.optim.Adam(tcn.parameters())
loss = torch.nn.CrossEntropyLoss()

one_hot = one_hot.to(device)

for iterations in tqdm(range(n_iterations)):
    # We sample with seq_len + 1 because of how we construct inputs and
    # labels (see below).
    batch = make_random_batch(batch_size, seq_len+1)
    # We will train the network to predict the next character in the sequence
    # That means our input is our batch of data except for the last character in
    # each sequence, and our labels are everything except the first character.
    batch_data = batch[:,:,:-1]
    # We use argmax(dim=1) to convert each label from a one-hot vector to a
    # class index.
    batch_label = batch[:,:,1:].argmax(dim=1)
    
    o = tcn(batch_data)
    loss_val = loss(o, batch_label)
    
    logger.add_scalar('train/loss', loss_val, global_step=iterations)
    
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
def sample(m, length=100):
    # S is the input we want to give to the network
    S = list("Model")
    for i in range(length):
        # Now we convert our input to a one-hot encoding
        data = torch.as_tensor(np.array(S)[None,:] == np.array(char_set)[:,None]).float()
        # m(data[None]) is the output of our model on a batch consisting only
        # of S, then m(data[None])[0,:,-1] is a distribution over the next
        # character we should predict.
        o = m(data[None])[0,:,-1]
        # Construct an actual distribution from the distrubtion parameters
        # returned by our network and then sample one character.
        s = torch.distributions.Categorical(logits=o).sample()
        # Now we add the character to our input for the next iteration.
        S.append(char_set[s])
    return "".join(S)

print( sample(tcn.cpu()) )

Models in hit > nonelf ` reque, AMh19x

:hidThonwerfe, exangable to fair ghere same.

modeld compollr Opf
